In [6]:
from __future__ import division, print_function, absolute_import

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import cv2
import os
import warnings
import sys
from PIL import Image
from timeit import time

from stereo_reconstruction import structure
from stereo_reconstruction import processor
from stereo_reconstruction.features import match_images

from yolo3_deepsort.yolo import YOLO
from yolo3_deepsort.deep_sort.detection import Detection
from yolo3_deepsort.deep_sort.tracker import Tracker
from yolo3_deepsort.tools import generate_detections as gdet
from yolo3_deepsort.tools.plot_utils import draw_one_box as draw_box
from yolo3_deepsort.deep_sort.detection import Detection as ddet
warnings.filterwarnings('ignore')

# Create Components

In [7]:
# create yolov3 detector
# yolo = YOLO(model_path = 'yolo3_deepsort/model_data/yolo_ori.h5',
#             classes_path = 'yolo3_deepsort/model_data/coco_classes.txt',
#             weights_only = True,
#             score = 0.3,
#             iou = 0.5) # coco version
yolo = YOLO(model_path = 'yolo3_deepsort/model_data/trained_weights_coco.h5',
            classes_path = 'yolo3_deepsort/model_data/classes_name.txt',
            weights_only = True,
            score = 0.3,
            iou = 0.5) # our version

# create feature encoder
model_filename = 'yolo3_deepsort/model_data/mars-small128.pb'
encoder = gdet.create_box_encoder(model_filename,batch_size=1)

# create tracker
nms_max_overlap = 1.0
max_distance = 0.3
lambda0 = 0.1
nn_budget = None
tracker = Tracker(metric_mode="cosine",max_cosine_distance=max_distance,
                      lambda0 = lambda0,nn_budget=nn_budget)

yolo3_deepsort/model_data/trained_weights_coco.h5 model, anchors, and classes loaded.


# Detection Tracking and Matching

In [5]:
video_path = r"G:\binocular_video\20191022_022500_video.h265"
cap = cv2.VideoCapture(video_path)
w = int(cap.get(3))
h = int(cap.get(4))
fps = cap.get(5)

out = cv2.VideoWriter('result\output.avi',cv2.VideoWriter_fourcc(*'MJPG'), fps, (w//2,2*h))
fontsize = 25

while cap.isOpened():
    # read video
    ret,frame = cap.read()
    
#     h,w,_ = frame.shape
    frame_left = frame[:,:w//2,:]
    frame_right = frame[:,:w//2,:]
    img_left_pil = Image.fromarray(frame_left.copy()[...,::-1])
    img_right_pil = Image.fromarray(frame_right.copy()[...,::-1])
    
    # detection
    boxes,classes,scores = yolo.detect_image(img_left_pil)
    
    # encoder features
    features = encoder(frame_left,boxes)
    detections = [Detection(bbox, score, feature,class_)
                    for bbox,score,feature,class_ in zip(boxes,scores,features,classes)]
    
    # call the tracker
    tracker.predict()
    tracker.update(detections)
    
    # Visualize result
    img_left_draw = img_left_pil.copy()
    img_right_draw = img_right_pil.copy()
    for i,track in enumerate(tracker.tracks):
        if not track.is_confirmed() or track.time_since_update > 1:
            continue
        bbox = track.to_tlbr()
        bbox_matched = match_images(frame_left,frame_right,bbox)# match two image
        
        # property of the object
        object_id = track.track_id
        object_class = track.object_class
        color = yolo.colors[yolo.class_names.index(track.object_class)]
        # draw boxes
        img_left_draw = draw_box(img_left_draw,bbox,object_id,object_class,color,fontsize)
        img_right_draw = draw_box(img_right_draw,bbox,object_id,object_class,color,fontsize)
        
    img_left_show = np.asarray(img_left_draw)
    img_right_show = np.asarray(img_right_draw)
    img_show = np.concatenate((img_left_show,img_right_show),axis = 0)
    cv2.imshow('demo',cv2.resize(img_show[...,::-1],(int(w/2.5),int(h/1.5))))
    
    out.write(img_show[...,::-1])# save result
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

out.release()
cap.release()
cv2.destroyAllWindows()